a) Целевая переменная price и log_price
 price и log_price неожиданно не сильно коррелируют между собой (0.42), но log_price сильнее и чище коррелирует 
 с признаками, поэтому лучше подходит для моделей (возможно нужно будет оставить ее)

 b) Наиболее сильные корреляты log_price (новой целевой переменной) => увеличение стоимости:
total_area              = 0.76   # Сильнейший коррелят. Ожидаемо — чем больше площадь, тем дороже
building_area_mean      = 0.74   # Размер здания связан с ценой
living_area             = 0.72   # Поддерживает тренд общей площади
rooms                   = 0.64   # Больше комнат → дороже
building_price_mean     = 0.59   # отражает среднюю цену по зданию, мощный фич
kitchen_area            = 0.57	 # Похожий эффект как на жилой и общей площади
ceiling_height          = 0.56   # Высокие потолки чаще в премиальных квартирах
ceiling_height_cluster  = 0.24   # Отражает распределение потолков — слабая +
living_ratio            = 0.20   # Незначительная положительная 
flor_total              = 0.18   # незначительная положительная
floor                   = 0.15   # Незначительно положительная
build_year              = 0.15   # Незначительно положительная
has_elevator            = 0.05   # Слабо влияет на прямую
floor_ratio	            = 0.05   # Может быть связано с типом квартиры (на первом/последнем этажах)
flats_count_cluster	    = 0.04	 # слабо влияет на прямую

 с) Отрицательные корреляции с log_price (новой целевой переменной) => уменьшение стоимости:
dist_origin_dest        = -0.42	 # Удалённость от центра города - возможно связано с тем, что это данные по Москве
distance_to_metro_fast  = -0.16  # Чем дальше от метро, тем дешевле
building_type_int       = -0.34  # тип здания влияет на снижении цены
kitchen_ratio           = -0.18  # слабая отрицательная корреляция с ценой (возможно, отражает непропорционально большие кухни в дешёвых квартирах).
age	                    = -0.15	 # Старые здания — чуть дешевле

 в данном датасете - мусорный рекивит - studio (так как везде = 0)

 d) Мультиколлинеарность — признаки с сильной корреляцией между собой
living_area - total_area                    = 0.95	# Почти одно и то же, можно оставить один
total_area - building_area_mean             = 0.85 # Очень сильно связаны, возможно лучше оставить только один
living_area - building_area_mean            = 0.80 # Очень сильно связаны, возможно лучше оставить только один
rooms - living_area                         = 0.75	# Сильно связаны, возможно лучше оставить только один
building_price_mean - building_area_mean    = 0.70	# Сильно связаны, возможно лучше оставить только один
total_area - kitchen_area                   = 0.67 # Логичная корреляция
total_area - rooms                          = 0.76 # Чем больше площадь, тем больше комнат
floor - floors_total                        = 0.61	# Более высокие этажи в высотных домах

 Нужно проверить не чувствительна ли модель к мультиколлинеарности.

 is_apartment, building_type_int и has_elevator слабо коррелируют с ценой — возможно, имеют категориальный/непрямой эффект.

 e) Промежуточные выводы: 
Для моделей оставить log_price как целевую переменную.
Провести отбор признаков, чтобы избежать мультиколлинеарности.
Использовать total_area, building_area_mean, living_area, rooms , building_price_mean, kitchen_area, ceiling_height, 
dist_origin_dest, distance_to_metro_fast, building_type_int как основу модели — они показывают сильную связь с ценой.
Категориальные признаки (build_type_floors, location_cluster, ceiling_height_cluster, flats_count_cluster) могут быть 
важны через one-hot или catboost encoding.

Зависимость цены (log_price) от is_apartment
 - is_apartment имеет слабую предсказательную силу.
 Можно использовать как вспомогательный бинарный признак — особенно в сочетании с floors_total или
 dist_origin_dest, distance_to_metro_fast (например, если далеко от центра города или метро  — это может быть минус).

 Зависимость цены (log_price) от has_elevator
 - has_elevator имеет слабую предсказательную силу.
 Можно использовать как вспомогательный бинарный признак — особенно в сочетании с location_cluster 
 или (например, если этажность > 5 и лифта нет — это может быть минус).

 Зависимость цены (log_price) от building_type_int
 - building_type_int имеет информативность и отражает 
 ценовой уровень района, можно использовать в моделе (через OneHotEncoding).

 Зависимость цены (log_price) от build_type_floors
 - build_type_floors имеет слабую предсказательную силу.

 Зависимость цены (log_price) от location_cluster
 - location_cluster имеет информативность и отражает ценовой уровень района, можно использовать 
 в моделе (через OneHotEncoding).

 Зависимость цены (log_price) от ceiling_height_cluster
 - ceiling_height_cluster имеет информативность и отражает ценовой уровень квартиры, можно использовать 
 в моделе (через OneHotEncoding).

 Зависимость цены (log_price) от flats_count_cluster
 - floor_group - слабоположительное влияние, примерно такое же как у living_area - вероятно оставлю в модели
 - build_type_floors и build_height_group - слабоположительное влияние, примерно одинаковые, но в корреляционном анализе
                                           влияние build_height_group сильнее 0.21 к 0.05, поэтому в модели оставлю его 
 - bad_floor_flag - слабоотрицательное влияние, но по RandomForest не важная фича, возможно уберу на последующих этапах.

 Наиболее значимые по версии RandomForest: building_price_mean, total_area, rooms, building_area_mean,
 living_ratio, living_area, dist_origin_dest, kitchen_ratio, floor_ratio, kitchen_area, floor,
 distance_to_metro_fast, kitchen_area

 Данные имеют асимметрию или длинные хвосты (например, в area).
 Варианты поступил так:
 Ограничил значения в некоторых столбцах (..._area) вручную при помощи clip, остальные оставил без изменения. 
 одели вроде CatBoost, RandomForest, Tree-based не боятся выбросов. Можно оставить как есть. В ручную 
 откорректировал мах высоту потолков (было 27 - стало 8 - если есть 2х уровневые квартиры)

Итого: 
 Для сравнения буду использовать 3 набора признаков:

 1. полный набор признаков (31)

 2. 
 - числовые: building_price_mean, total_area, rooms, building_area_mean,
 living_ratio, living_area, dist_origin_dest, kitchen_ratio, floor_ratio, kitchen_area, floor,
 distance_to_metro_fast+
 - категории: is_apartment, has_elevator, building_type_int, build_type_floors, location_cluster, 
 ceiling_height_cluster, flats_count_cluster, build_height_group, floor_group (21)

 3. 
 - отобрал по карте корреляционного анализа: kitchen_area, living_area, rooms, total_area, ceiling_height
 building_price_mean, building_area_mean, ceiling_height_cluster, build_height_group, living_ratio (положительное влияние),
 building_type_int, dist_origin_dest, distance_to_metro_fast, age (отрицательное влияние) (14)

 MAE модели с первым набором фичей = 3532582.674518879
 MAE модели со вторым набором фичей = 3587201.5842803796
 MAE модели с третьим набором фичей = 3627241.0572573096

 MASE (2 vs 1) = 1.0154614668059934 => MASE > 1, модель на втором сете признаков работает хуже.
 MASE (3 vs 2) = 1.0111617571625717 => MASE > 1, модель на третьем сете признаков работает хуже второй.
 MASE (3 vs 1) = 1.0267958011064307 => MASE > 1, модель на третьем сете признаков работает хуже первой и второй => худшая из 3х.

 Следовательно за базовую модель возьмем модель на ПЕРВОМ сэте признаков.